In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Chris dataset
df = pd.read_csv('/Users/renatoboemer/code/boemer00/Netflix/raw_data/merged_movies_by_index.csv')

In [5]:
df.shape  # 80% = 8309 |  20% = 2077  

(10386, 34)

## Data Split

In [6]:
df = df[:8309]
df_test = df[8309:]

In [7]:
df.shape

(8309, 34)

In [8]:
df.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test_DONT_TOUCH.csv', index=False)

## Data Wrangling

In [9]:
def data_wrangling(df):
    """ cleaning irrelevant rows and columns """ 
    
    # drop irrelevant columns
    df = df.drop(columns=['title', 'year', 'Awards', 'Poster', 'Metascore', 'DVD',
                          'BoxOffice', 'Internet Movie Database','totalSeasons',
                          'imdbVotes','Website', 'Response', 'Production', 'Metacritic', 'Ratings'])

    ## fill nan and' min', convert to int and replace zero for the mean
    df['Runtime'] = df['Runtime'].fillna(0).apply(lambda x: str(x).replace(',', ''))
    df['Runtime'] = df['Runtime'].apply(lambda x: float(str(x).replace(' min', '')))
    df['Runtime'] = df['Runtime'].replace(0, df['Runtime'].mean())
    
    ## fill nan and remove '%', convert to float and replace zero for the mean
    df['Rotten Tomatoes'] = df['Rotten Tomatoes'].fillna(0) 
    df['Rotten Tomatoes'] = df['Rotten Tomatoes'].apply(lambda x: float(str(x).replace('%', '')))
    df['Rotten Tomatoes'] = df['Rotten Tomatoes'].replace(0, df['Rotten Tomatoes'].mean())
    
    ## dropna
    df = df.dropna(subset = ['Actors', 'Director', 'Writer', 'Language'])
    
    ## replace with other frequent values
    freq_country = df[['Country']].value_counts().reset_index()['Country'][0]
    df['Country'] = df['Country'].replace(0, freq_country).replace('United States', freq_country)
    freq_genre = df['Genre'].mode()[0]
    df['Genre'] = df['Genre'].replace(np.nan, freq_genre)
    df['Plot'] = df['Plot'].replace(np.nan,'unknown')
    
    return df
    

In [10]:
# questions: index_match?
df = data_wrangling(df)

In [11]:
df.shape

(7309, 19)

## Data Exploration

## Pipeline

## MLFlow

In [15]:
from mlflow.tracking import MlflowClient

EXPERIMENT_NAME = "[UK] [London] [boemer] netflix + v1"
client = MlflowClient()
try:
    experiment_id = client.create_experiment(EXPERIMENT_NAME)
except BaseException:
    experiment_id = client.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

for model in ["linear", "Randomforest"]:
    run = client.create_run(experiment_id)
    client.log_metric(run.info.run_id, "rmse", 4.5)
    client.log_param(run.info.run_id, "model", model)

In [14]:
!mlflow ui

[2021-06-10 14:16:26 +0100] [85034] [INFO] Starting gunicorn 20.1.0
[2021-06-10 14:16:26 +0100] [85034] [INFO] Listening at: http://127.0.0.1:5000 (85034)
[2021-06-10 14:16:26 +0100] [85034] [INFO] Using worker: sync
[2021-06-10 14:16:26 +0100] [85116] [INFO] Booting worker with pid: 85116
^C
[2021-06-10 14:18:28 +0100] [85034] [INFO] Handling signal: int
[2021-06-10 14:18:28 +0100] [85116] [INFO] Worker exiting (pid: 85116)
